In [28]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from config import database, user, password, host, port
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry
import geopandas as gpd
import pandas as pd
from shapely import wkt
import psycopg2

In [29]:
# import packages
import requests
import json

In [30]:
# send the request
def jason2dataf(url):
    response = requests.get(url)

    # store the raw text of the response in a variable
    raw_data = response.text

    # parse the raw text response into a JSON
    data = json.loads(raw_data)

    # from JSON to Pandas DataFrame
    data_df = pd.json_normalize(data)
    return data_df

In [31]:
# Fetch and convert data from the ISPRA Idrogeo API for the Lombardy region and its provinces into DataFrames.
# The jason2dataf function takes a URL endpoint as an argument, makes a request to the API,
# retrieves the JSON data, and converts it into a pandas DataFrame for further analysis.
dataf_lombardia = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/regioni/3')
data_prov_milan = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/15')
data_prov_varese = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/12')
data_prov_como = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/13')
data_prov_lecco = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/97')
data_prov_sondrio = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/14')
data_prov_bergamo = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/16')
data_prov_brescia = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/17')
data_prov_pavia = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/18')
data_prov_cremona = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/19')
data_prov_mantova = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/20')
data_prov_lodi = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/98')
data_prov_Monza = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/108')



In [32]:
# Merge the DataFrames
merged_df = pd.concat([data_prov_milan, data_prov_varese,data_prov_como,data_prov_lecco,data_prov_sondrio,data_prov_bergamo,data_prov_brescia,data_prov_pavia,data_prov_cremona,data_prov_mantova,data_prov_lodi,data_prov_Monza]).reset_index(drop=True)

# Rename the 'cod_prov' column to 'COD_PROV' to maintain consistency in column naming conventions.
merged_df.rename(columns={'cod_prov':'COD_PROV'}, inplace=True)


In [33]:
# Define the file paths for the shapefiles of the region and provinces.
# These shapefiles contain the geographical boundaries for the Lombardy region and its provinces.
shapefile_path_region = "./Limiti01012024_g/Reg01012024_g/Reg01012024_g_WGS84.shp"
shapefile_path_province = "./Limiti01012024_g/ProvCM01012024_g/ProvCM01012024_g_WGS84.shp"

# Use Geopandas to read the shapefiles into GeoDataFrames.
gdf_region = gpd.read_file(shapefile_path_region)
gdf_province = gpd.read_file(shapefile_path_province)


In [34]:

# Define the list of desired province codes (COD_PROV) for filtering.
# These codes correspond to the provinces within the Lombardy region that we are interested in.
desired_cod_RIP = [15, 12, 13, 97, 14, 16, 17, 18, 19, 20, 98, 108]

# Filter the GeoDataFrame based on the desired cod_RIP values
filtered_gdf = gdf_province[gdf_province['COD_PROV'].isin(desired_cod_RIP)]



In [35]:

# Separate the 'geometry' column from the filtered GeoDataFrame
geometry = filtered_gdf.pop('geometry')

# Create a new GeoDataFrame by combining the filtered data with the 'geometry' column
filtered_gdf3 = gpd.GeoDataFrame(filtered_gdf, geometry=geometry)

# Merge the new GeoDataFrame with the previously merged DataFrame on the 'COD_PROV' column
# Use a left join to ensure all rows from the GeoDataFrame are kept, adding data from the merged DataFrame
merged_df3 = pd.merge(filtered_gdf3, merged_df, on='COD_PROV', how='left')


In [36]:
# Create a GeoDataFrame from the merged DataFrame, specifying the 'geometry' column
merged_gdf = gpd.GeoDataFrame(merged_df3, geometry='geometry')
merged_gdf

,COD_RIP,COD_REG,COD_PROV,COD_CM,COD_UTS,DEN_PROV,DEN_CM,DEN_UTS,SIGLA,TIPO_UTS,...,bbcc_fr_aa,bbccfrp3p4,bbccfrp4_p,bbccfrp3_p,bbccfrp2_p,bbccfrp1_p,bbccfraa_p,bbccfrp34p,cod_rip,cod_reg
0,1,3,12,0,12,Varese,-,Varese,VA,Provincia,...,0,17,0.817,0.572,3.350,0,0,1.389,1,3
1,1,3,13,0,13,Como,-,Como,CO,Provincia,...,0,78,2.592,2.729,10.095,0,0,5.321,1,3
2,1,3,14,0,14,Sondrio,-,Sondrio,SO,Provincia,...,0,49,2.022,2.932,45.399,0,0,4.954,1,3
3,1,3,15,215,215,-,Milano,Milano,MI,Città metropolitana,...,0,0,0.000,0.000,0.000,0,0,0.000,1,3
4,1,3,16,0,16,Bergamo,-,Bergamo,BG,Provincia,...,0,92,1.653,1.727,7.825,0,0,3.380,1,3
5,1,3,17,0,17,Brescia,-,Brescia,BS,Provincia,...,0,104,1.144,1.987,10.509,0,0,3.132,1,3
6,1,3,18,0,18,Pavia,-,Pavia,PV,Provincia,...,0,17,0.377,0.691,1.257,0,0,1.069,1,3
7,1,3,19,0,19,Cremona,-,Cremona,CR,Provincia,...,0,0,0.000,0.000,0.000,0,0,0.000,1,3
8,1,3,20,0,20,Mantova,-,Mantova,MN,Provincia,...,0,0,0.000,0.000,0.000,0,0,0.000,1,3
9,1,3,97,0,97,Lecco,-,Lecco,LC,Provincia,...,0,40,2.061,2.788,16.242,0,0,4.848,1,3


In [37]:
# Define the list of columns to select from the merged GeoDataFrame
columns_to_select =  ['geometry','cod_reg','cod_rip','COD_PROV','ar_kmq','nome','uid',
    'ar_fr_p3p4', 'ar_fr_p2', 'ar_fr_p1', 'ar_fr_p3' , 'ar_fr_p4' , 'ar_fr_aa', 'ar_frp4_p','ar_frp3_p', 'ar_frp2_p', 
    'ar_frp1_p','ar_fraa_p', 'ar_frp3p4p',
    'pop_fr_p2', 'pop_fr_p1', 'pop_fr_p3', 'pop_fr_p4', 'pop_fr_aa','popfrp4_p','popfrp3_p','popfrp2_p','popfrp1_p', 
    'popfrp3p4p', 'popfr_p3p4' , 'popfraa_p', 'ed_fr_p3', 'ed_fr_p4',
    'ed_fr_p2', 'ed_fr_p1', 
    'ed_fr_p3p4', 'edfrp3p4p', 
    'fam_tot', 'fam_idr_p3', 'fam_idr_p2', 'fam_idr_p1', 'famidp3_p', 'famidp2_p', 'famidp1_p', 'fam_fr_p4', 'fam_fr_p3', 'fam_fr_p2', 'fam_fr_p1',  'fam_fr_aa', 'famfr_p3p4', 'famfrp4_p', 'famfrp3_p', 'famfrp2_p', 'famfrp1_p', 'famfraa_p', 'famfrp3p4p'
]

# Select the specified columns from the merged GeoDataFrame
selected_df = merged_gdf[columns_to_select]

# Create a copy of the selected DataFrame for further processing
selected_df2 = selected_df.copy()


In [39]:
# # Connect to the default database to create the new database
# conn = psycopg2.connect(host=host,
#     port=port,
#     dbname=database,
#     user=user,
#     password=password)
# # conn.autocommit = True # needed to create a db programmatically
# cursor = conn.cursor()
# # Create the new database
# try:
#     cursor.execute(f"CREATE DATABASE {database};")
# except:
#     pass  # If the database already exists, ignore the error
# 
# # Close the connection to the default database
# conn.close()

In [40]:
# Connect to the database (replace with your actual database URI)
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:5432/{database}')
Session = sessionmaker(bind=engine)
session = Session()


In [42]:
from shapely.wkb import loads as wkb_loads
from shapely.wkt import dumps as wkt_dumps
from shapely.geometry import MultiPolygon, Polygon
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text, create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import declarative_base

# Define the Base class for SQLAlchemy
Base = declarative_base()

# Define your SQLAlchemy model
class Dataset(Base):
    __tablename__ = 'dataset'

    id = Column(Integer, primary_key=True)
    cod_reg = Column(Integer)
    cod_rip = Column(Integer)
    cod_prov = Column(Integer)
    ar_kmq = Column(Float)
    nome = Column(String)
    uid = Column(Integer)
    ar_fr_p3p4 = Column(Float)
    ar_fr_p2 = Column(Float)
    ar_fr_p1 = Column(Float)
    ar_fr_p3 = Column(Float)
    ar_fr_p4 = Column(Float)
    ar_fr_aa = Column(Float)
    ar_frp3p4p = Column(Float)
    ar_frp4_p = Column(Float)
    ar_frp3_p = Column(Float)
    ar_frp2_p = Column(Float)
    ar_frp1_p = Column(Float)
    ar_fraa_p = Column(Float)
    pop_fr_p2 = Column(Float)
    pop_fr_p1 = Column(Float)
    pop_fr_p3 = Column(Float)
    pop_fr_p4 = Column(Float)
    popfr_p3p4 = Column(Float)
    pop_fr_aa = Column(Float)
    popfrp4_p = Column(Float)
    popfrp3_p = Column(Float)
    popfrp2_p = Column(Float)
    popfrp1_p = Column(Float)
    popfrp3p4p = Column(Float)
    popfraa_p = Column(Float)
    ed_fr_p4 = Column(Float)
    ed_fr_p3 = Column(Float)
    ed_fr_p2 = Column(Float)
    ed_fr_p1 = Column(Float)
    ed_fr_p3p4 = Column(Float)
    edfrp3p4p = Column(Float)
    # Family
    fam_tot = Column(Float)
    fam_idr_p3 = Column(Float)
    fam_idr_p2 = Column(Float)
    fam_idr_p1 = Column(Float)
    famidp3_p = Column(Float)
    famidp2_p = Column(Float)
    famidp1_p = Column(Float)
    fam_fr_p4 = Column(Float)
    fam_fr_p3 = Column(Float)
    fam_fr_p2 = Column(Float)
    fam_fr_p1 = Column(Float)
    fam_fr_aa = Column(Float)
    famfr_p3p4 = Column(Float)
    famfrp4_p = Column(Float)
    famfrp3_p = Column(Float)
    famfrp2_p = Column(Float)
    famfrp1_p = Column(Float)
    famfraa_p = Column(Float)
    famfrp3p4p = Column(Float)
    geometry = Column(Text)  # Assuming you're storing geometry as WKT in a text column

    def __repr__(self):
        return f"<Dataset(id={self.id}, nome='{self.nome}')>" # String representation for debugging

In [43]:
# Function to convert WKB to WKT
def convert_wkb_to_wkt(geom):
    if isinstance(geom, str):
        geom = bytes.fromhex(geom)  # Convert hex string to bytes
    if isinstance(geom, (bytes, bytearray)):
        return wkt_dumps(wkb_loads(geom))  # Convert WKB to Shapely geometry, then to WKT
    elif isinstance(geom, (MultiPolygon, Polygon)):
        return wkt_dumps(geom)  # If it's already a Shapely geometry, convert directly to WKT
    else:
        raise TypeError(f"Unexpected geometry type: {type(geom)}")

# Apply conversion function to the 'geometry' column
selected_df2['geometry_wkt'] = selected_df2['geometry'].apply(convert_wkb_to_wkt)

# Connect to the database (replace with your actual database URI)
# Create an engine and connect to the PostgreSQL database
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:5432/{database}')

# Create the table
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Iterate over the rows in the DataFrame and insert them into the database
for index, row in selected_df2.iterrows():
    dataset_entry = Dataset(
         cod_reg=row['cod_reg'],
        cod_rip=row['cod_rip'],
        cod_prov=row['COD_PROV'],
        ar_kmq=row['ar_kmq'],
        nome=row['nome'],
        uid=row['uid'],
        ar_fr_p3p4=row['ar_fr_p3p4'],
        ar_fr_p2=row['ar_fr_p2'],
        ar_fr_p1=row['ar_fr_p1'],
        ar_fr_p3=row['ar_fr_p3'],
        ar_fr_p4=row['ar_fr_p4'],
        ar_fr_aa=row['ar_fr_aa'],
        ar_frp3p4p=row['ar_frp3p4p'],
        ar_frp4_p=row['ar_frp4_p'],
        ar_frp3_p=row['ar_frp3_p'],
        ar_frp2_p=row['ar_frp2_p'],
        ar_frp1_p=row['ar_frp1_p'],
        ar_fraa_p=row['ar_fraa_p'],
        pop_fr_p2=row['pop_fr_p2'],
        pop_fr_p1=row['pop_fr_p1'],
        pop_fr_p3=row['pop_fr_p3'],
        pop_fr_p4=row['pop_fr_p4'],
        popfr_p3p4=row['popfr_p3p4'],
        pop_fr_aa=row['pop_fr_aa'],
        popfrp4_p=row['popfrp4_p'],
        popfrp3_p=row['popfrp3_p'],
        popfrp2_p=row['popfrp2_p'],
        popfrp1_p=row['popfrp1_p'],
        popfrp3p4p=row['popfrp3p4p'],
        popfraa_p=row['popfraa_p'],
        ed_fr_p4=row['ed_fr_p4'],
        ed_fr_p3=row['ed_fr_p3'],
        ed_fr_p2=row['ed_fr_p2'],
        ed_fr_p1=row['ed_fr_p1'],
        ed_fr_p3p4=row['ed_fr_p3p4'],
        edfrp3p4p=row['edfrp3p4p'],
        #Family
        fam_tot=row['fam_tot'],
        fam_idr_p3=row['fam_idr_p3'],
        fam_idr_p2=row['fam_idr_p2'],
        fam_idr_p1=row['fam_idr_p1'],
        famidp3_p=row['famidp3_p'],
        famidp2_p=row['famidp2_p'],
        famidp1_p=row['famidp1_p'],
        fam_fr_p4=row['fam_fr_p4'],
        fam_fr_p3=row['fam_fr_p3'],
        fam_fr_p2=row['fam_fr_p2'],
        fam_fr_p1=row['fam_fr_p1'],
        fam_fr_aa=row['fam_fr_aa'],
        famfr_p3p4=row['famfr_p3p4'],
        famfrp4_p=row['famfrp4_p'],
        famfrp3_p=row['famfrp3_p'],
        famfrp2_p=row['famfrp2_p'],
        famfrp1_p=row['famfrp1_p'],
        famfraa_p=row['famfraa_p'],
        famfrp3p4p=row['famfrp3p4p'],
        geometry=row['geometry_wkt']  # Insert WKT geometry
    )
    session.add(dataset_entry)


# Commit the session to save the changes in the database
try:
    session.commit()
except Exception as e:
    session.rollback() # Rollback the session in case of an error
    print(f"Error committing session: {e}")
finally:
    session.close() # Close the session